# Slope and aspect
We have the MERIT DEM showing elevation. From this we calculate slope and aspect on the pixel level.

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import _functions as pmf

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [3]:
# Get the required info from the config file
raw_path = pmf.read_from_config(config_file,'raw_path')
temp_path = pmf.read_from_config(config_file,'temp_path')
tar_crs =  pmf.read_from_config(config_file,'topo_crs')

### 1. File location

In [4]:
import glob

In [5]:
src_path = Path(raw_path) / 'merit' / 'raw'

In [6]:
dem_path = sorted( glob.glob(str(src_path / '*.tif')) )

In [7]:
dem_path = str(dem_path[0]) # Get out of list, convert to string for GDAL

In [8]:
dem_path

'/Users/wmk934/data/NorthAmerica_geospatial/merit/raw/merit_hydro_elv.tif'

### 2. Create the output directory

In [9]:
temp_path = Path(temp_path) / 'merit'

In [10]:
slope_path = Path(raw_path) / 'merit' / 'slope'
aspect_path = Path(raw_path) / 'merit' / 'aspect'

In [11]:
slope_path.mkdir(exist_ok=True, parents=True)
aspect_path.mkdir(exist_ok=True, parents=True)
temp_path.mkdir(exist_ok=True, parents=True)

### 3. Convert the DEM into a projection in meters
We need this because calcualting a slope on in EPSG:4326 returns wonky results.

In [11]:
from osgeo import gdal

In [12]:
gdal.UseExceptions()

In [14]:
tmp = tar_crs.replace(':','-') # because f-strings choke on this
lcc_elev = str(temp_path / f'merit_hydro_elv_{tmp}.tif')

In [15]:
options = gdal.WarpOptions(format='GTiff',
                           srcSRS='epsg:4326', dstSRS=tar_crs)#,
                           #creationOptions=['COMPRESS=DEFLATE', 'BIGTIFF=YES'])

In [16]:
gdal.Warp(lcc_elev, dem_path, options=options)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x1043221f0> >

### 4. Create slope and aspect files
We'll run this through subprocess because for some reason I can't get `gdal.DEMProcessing` to work as expected.

In [17]:
import subprocess

### 4.1 Slope

In [18]:
lcc_slope = str(temp_path / 'merit_hydro_slope.tif') # Still in ESRI:102009

In [19]:
gdal_command = ['gdaldem', 'slope', lcc_elev, lcc_slope, 
                '-alg', 'Horn', '-compute_edges']#,
                #'-co', 'COMPRESS=DEFLATE', '-co', 'BIGTIFF=YES']

In [20]:
subprocess.run(gdal_command)

0...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args=['gdaldem', 'slope', '/Users/wmk934/data/perceptual_models/tmp/merit/merit_hydro_elv_ESRI-102009.tif', '/Users/wmk934/data/perceptual_models/tmp/merit/merit_hydro_slope.tif', '-alg', 'Horn', '-compute_edges'], returncode=0)

### 4.2 Aspect

In [113]:
lcc_aspect = str(temp_path / 'merit_hydro_aspect.tif') # Still in ESRI:102009

In [111]:
gdal_command = ['gdaldem', 'aspect', lcc_elev, lcc_aspect, 
                '-alg', 'Horn', '-compute_edges']#,
                #'-co', 'COMPRESS=DEFLATE', '-co', 'BIGTIFF=YES']

In [112]:
subprocess.run(gdal_command)

0...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args=['gdaldem', 'aspect', '/Users/wmk934/data/perceptual_models/tmp/merit/merit_hydro_elv_ESRI-102009.tif', '/Users/wmk934/data/perceptual_models/tmp/merit/merit_hydro_slope_sub.tif', '-alg', 'Horn', '-compute_edges'], returncode=0)

## 5. Transform back to EPSG:4326 and compress

In [13]:
warp_options = gdal.WarpOptions(format='GTiff',srcSRS=tar_crs, dstSRS='epsg:4326')
tran_options = ['COMPRESS=DEFLATE', 'BIGTIFF=YES']

In [14]:
tmp_file = str(temp_path / 'transfer.tif')

### 5.1 Slope

In [16]:
slope_out = str(slope_path / 'merit_hydro_slope.tif')

In [24]:
gdal.Warp(tmp_file, lcc_slope, options=warp_options).FlushCache() # Reproject

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x1042abc90> >

In [19]:
gdal.Translate(slope_out, tmp_file, creationOptions=tran_options).FlushCache()

0

### 5.2 Aspect

In [120]:
aspect_out = str(aspect_path / 'merit_hydro_aspect.tif')

In [121]:
gdal.Warp(tmp_file, lcc_aspect, options=warp_options).FlushCache() # Reproject

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x103c91ce0> >

In [122]:
gdal.Translate(aspect_out, tmp_file, creationOptions=tran_options).FlushCache()

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x103c91200> >

## 6. Remove the temp folder

In [20]:
import shutil

In [21]:
shutil.rmtree(temp_path)